In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv("../input/train.csv")
print("train_data shape: ", train_data.shape)
train_data.head()

In [ ]:
train_data["Category"].unique()


In [ ]:
#df.drop(df.columns[[0, 1, 3]], axis=1)
X = train_data.drop(train_data.columns[[1, 2, 5, 6]], axis = 1)
X.head()

In [ ]:
y = train_data.iloc[:, 1]
y.head()

In [ ]:
mapping_week= {'Monday':0, 'Tuesday':0, 'Wednesday':0, 'Thursday':0, 'Friday':0, 'Saturday':1, 'Sunday':1}


In [ ]:
def preprocess_data(dataset):
    dataset['Dates'] = pd.to_datetime(dataset['Dates'])
    dataset['Month'] = dataset.Dates.apply(lambda x: x.month)
    dataset['Hour'] = dataset.Dates.apply(lambda x: x.hour)

    # Extract Feature for Third Dimension
    #dataset['x'] = dataset.apply(lambda x: np.cos(x['Y']) * np.cos(x['X']), axis = 1)
    #dataset['y'] = dataset.apply(lambda x: np.cos(x['Y']) * np.sin(x['X']), axis = 1)
    #dataset['z'] = dataset.apply(lambda x: np.sin(x['Y']), axis = 1)
    
    dataset['DayOfWeek'] = dataset['DayOfWeek'].map(mapping_week)
   
    dataset['Morning'] = dataset['Hour'].map(lambda x: 1 if 5 <= x < 12 else 0)
    dataset['Afternoon'] = dataset['Hour'].map(lambda x: 1 if 12 <= x < 18 else 0 )
    dataset['Evening'] = dataset['Hour'].map(lambda x: 1 if 18 <= x < 22 else 0 )
    dataset['Night'] = dataset['Hour'].map(lambda x: 1 if 22 <= x < 24 or 0 <= x < 5 else 0 )

    # spring=3-5 | summer = 6-8 | fall = 9-11 | Winter = 12-2
    dataset['Spring'] = dataset['Month'].map(lambda x: 1 if 3 <= x < 6 else 0)
    dataset['Summer'] = dataset['Month'].map(lambda x: 1 if 6 <= x < 9 else 0 )
    dataset['Fall'] = dataset['Month'].map(lambda x: 1 if 9 <= x < 12 else 0 )
    dataset['Winter'] = dataset['Month'].map(lambda x: 1 if x == 12 or 1 <= x < 3 else 0 )

    dataset = pd.get_dummies(data=dataset, columns=['PdDistrict'])
    return dataset

In [ ]:
X = preprocess_data(X)

In [ ]:
# Clean columns
X = X.drop('Dates', 1)
X = X.drop('Hour', 1)
X = X.drop('Month', 1)
# = X.drop('X', 1)
#X = X.drop('Y', 1)

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
y = y.to_frame()
le = LabelEncoder()
y["Category"] = le.fit_transform(y["Category"])
y.head()

In [ ]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categorical_features = [0])
y = enc.fit_transform(y).toarray()

In [ ]:
print(type(X), type(y))

In [ ]:
# Convert dataframe to ndarray
X = X.values

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
# ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
model_ann = Sequential()
model_ann.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
model_ann.add(Dropout(0.1))

model_ann.add(Dense(1024, activation='relu'))
model_ann.add(Dropout(0.1))

model_ann.add(Dense(1024, activation='relu'))
model_ann.add(Dropout(0.1))

model_ann.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
# Compiling the ANN
model_ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_ann.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 128, epochs = 25)



In [ ]:
y_pred = model_ann.predict(X_test)

In [ ]:
score , acc = model_ann.evaluate(X_test, y_test, batch_size = 128)

print('Log Loss score: ', score)
print('Test accuracy: ', acc)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

print(y_test.argmax(axis=1), y_test.shape)
print(y_pred.argmax(axis=1), y_pred.shape)

cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
cm = pd.DataFrame(cm)
sns.heatmap(cm, cmap="YlGnBu", square=True)

In [ ]:
test_data = pd.read_csv('../input/test.csv')
test_data.head()

In [ ]:
test_data = preprocess_data(test_data)
test_data.head()

In [ ]:
# Clean columns
test_data = test_data.drop('Id', 1)
test_data = test_data.drop('Address', 1)
test_data = test_data.drop('Dates', 1)
test_data = test_data.drop('Hour', 1)
test_data = test_data.drop('Month', 1)
#test_data = test_data.drop('X', 1)
#test_data = test_data.drop('Y', 1)
test_data.head()

In [ ]:
# Feature Scaling 
test_data = sc.transform(test_data)

In [ ]:
y_pred_proba = model_ann.predict_proba(test_data)

In [ ]:
import gzip
import csv
labels = 'ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS'.split(',')
with gzip.open('ann_predict.csv.gz', 'wt') as outf:
  fo = csv.writer(outf, lineterminator='\n')
  fo.writerow(['Id'] + labels)

  for i, pred in enumerate(y_pred_proba):
    fo.writerow([i] + list(pred))